**TRAIN PIPELINE**

In [1]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 16.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 100.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: 

In [2]:
!pip install -U spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 55.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.5
    Uninstalling spacy-3.8.5:
      Successfully uninstalled spacy-3.8.5


In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
!nvidia-smi

Tue May 20 13:31:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
cv_data = json.load(open("/kaggle/input/resumedataset/train_data.json"))

In [6]:
!python -m spacy init fill-config /kaggle/input/resumedataset/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot["entities"]
        ents = []
        entity_indices = []
        for start, end, label in annot:
          skip_entity = False
          for idx in range(start, end):
            if idx in entity_indices:
              print('TRUE',idx)
              skip_entity = True
              break
          if skip_entity == True:
            continue
          entity_indices = entity_indices + list(range(start, end))
          try:
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
          except:
            continue
          if span is None:
            err_data = str([start, end]) + " " + str(text) + "\n"
            file.write(err_data)
          else:
            ents.append(span)
        try:
          doc.ents = ents
          db.add(doc)
        except:
          pass
    return db

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [9]:
len(train), len(test)

(140, 60)

In [10]:
file = open("error_data.txt", "w")
db = get_spacy_doc(file, train)
db.to_disk("train_data.spacy")

db = get_spacy_doc(file, test)
db.to_disk("test_data.spacy")

file.close()

 16%|█▌        | 22/140 [00:00<00:01, 100.40it/s]

TRUE 1514
TRUE 6682
TRUE 823
TRUE 21
TRUE 3535
TRUE 3385
TRUE 52


 41%|████      | 57/140 [00:00<00:00, 101.45it/s]

TRUE 2733
TRUE 1180
TRUE 13
TRUE 549
TRUE 26
TRUE 463
TRUE 116
TRUE 15
TRUE 295
TRUE 729
TRUE 983
TRUE 67
TRUE 3


 65%|██████▌   | 91/140 [00:00<00:00, 100.00it/s]

...

TRUE 497
TRUE 381
TRUE 4708
TRUE 15


 67%|██████▋   | 40/60 [00:00<00:00, 94.50it/s]

TRUE 4374
TRUE 3132
TRUE 2665
TRUE 2562
TRUE 2407
TRUE 2184
TRUE 556
TRUE 263
TRUE 60
TRUE 3461
TRUE 34
TRUE 1106
TRUE 996
TRUE 43
TRUE 2886
TRUE 1161
TRUE 1576
TRUE 0


100%|██████████| 60/60 [00:00<00:00, 99.04it/s] 

TRUE 2076
TRUE 1865


In [11]:
with open("/kaggle/working/config.cfg", "r") as f:
    config_text = f.read()
print(config_text)


[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = "pytorch"
seed = 0

[nlp]
lang = "en"
pipeline = ["transformer","ner"]
batch_size = 128
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}
vectors = {"@vectors":"spacy.Vectors.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"

[components.transformer]
factory = "transformer"
max_batch_items = 4096
set_extra_annotations = {"@annotation_setters":"sp

**CUSTOM CONFIG**

In [13]:
config_text = config_text.replace("max_steps = 20000", "max_steps = 40000")
config_text = config_text.replace("patience = 1600", "patience = 3000")
config_text = config_text.replace("dropout = 0.1", "dropout = 0.05")
config_text = config_text.replace("use_averages = false", "use_averages = true")
config_text = config_text.replace("@schedules = \"warmup_linear.v1\"", "@schedules = \"warmup_cosine.v1\"")
config_text = config_text.replace("warmup_steps = 250", "warmup_steps = 500")
config_text = config_text.replace("total_steps = 20000", "total_steps = 40000")

In [14]:
print(config_text)

[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = "pytorch"
seed = 0

[nlp]
lang = "en"
pipeline = ["transformer","ner"]
batch_size = 128
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}
vectors = {"@vectors":"spacy.Vectors.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"

[components.transformer]
factory = "transformer"
max_batch_items = 4096
set_extra_annotations = {"@annotation_setters":"sp

In [15]:
!python -m spacy train /kaggle/working/config.cfg \
--output /kaggle/working/output \
--paths.train /kaggle/working/train_data.spacy \
--paths.dev /kaggle/working/test_data.spacy \
--gpu-id 0


✔ Created output directory: /kaggle/working/output
ℹ Saving to output directory: /kaggle/working/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2025-05-20 13:45:45,878] [INFO] Set up nlp object from config
[2025-05-20 13:45:45,897] [INFO] Pipeline: ['transformer', 'ner']
[2025-05-20 13:45:45,900] [INFO] Created vocabulary
[2025-05-20 13:45:45,900] [INFO] Finished initializing nlp object
tokenizer_config.json: 100%|██████████████████| 25.0/25.0 [00:00<00:00, 154kB/s]
config.json: 100%|█████████████████████████████| 481/481 [00:00<00:00, 6.00MB/s]
vocab.json: 100%|████████████████████████████| 899k/899k [00:00<00:00, 15.2MB/s]
merges.txt: 100%|████████████████████████████| 456k/456k [00:00<00:00, 7.80MB/s]
tokenizer.json: 100%|██████████████████████| 1.36M/1.36M [00:00<00:00, 21.7MB/s]
2025-05-20 13:45:52.500558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register fa

**MODEL EVALUATION**

In [29]:
nlp = spacy.load("/kaggle/working/output/model-best")

In [30]:
import os
import subprocess
from IPython.display import FileLink, display

def download_folder(folder_path, zip_name="downloaded_output"):
    # Ensure absolute path and safe folder name
    full_folder_path = os.path.abspath(folder_path)
    zip_file_path = f"/kaggle/working/{zip_name}.zip"

    # Use subprocess to zip the folder
    command = f"zip -r {zip_file_path} {full_folder_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print("❌ Failed to zip folder.")
        print(result.stderr)
    else:
        print("✅ Folder zipped successfully. Click the link below to download:")
        display(FileLink(f"{zip_name}.zip"))


In [35]:
!zip -r file.zip /kaggle/working



  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/train_data.spacy
zip I/O error: No space left on device
zip error: Output file write failure (write error on zip file)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
doc = nlp('Email: john.doe@example.com • Phone: +1 (555) 123‑4567 • LinkedIn: linkedin.com/in/johndoe

Professional Summary
Results‑driven Software Engineer with 3+ years of experience in backend development using Python and Node.js. Proven track record of designing RESTful APIs, optimizing database performance, and deploying containerized applications 
Indeed
. Passionate about clean code, test‑driven development, and continuous learning 
Novorésumé
.

Work Experience
Backend Developer, Tech Solutions Inc., San Francisco, CA
June 2021 – Present

Designed and implemented microservices architecture for e‑commerce platform, reducing response time by 30% 
Indeed
.

Developed RESTful APIs in Flask and Express.js, serving 10,000+ daily active users with 99.9% uptime.

Integrated PostgreSQL and Redis caching, achieving 40% faster data retrieval.

Wrote unit and integration tests (pytest, Mocha), raising code coverage to 85% 
Mignone Career Center
.

Junior Software Engineer, Innovatech Corp., Remote
July 2019 – May 2021

Collaborated in a 5‑member Agile team to build a real‑time analytics dashboard with Django and React.

Implemented user authentication (JWT) and role‑based access control, supporting 3 user tiers.

Automated CI/CD pipeline using GitHub Actions and Docker, cutting deployment time from 2 hours to 15 minutes.

')
for ent in doc.ents:
    print(ent.text, "  >>>", ent.label_)

In [ ]:
!pip install PyMuPDF

In [ ]:
import sys, fitz